https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

In [1]:
# on colab
!pip install langchain tiktoken chromadb pypdf transformers InstructorEmbedding accelerate bitsandbytes sentencepiece Xformers pdf2image sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 120.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 9.1 MB/s eta 0:0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/LLM_chatbot_devs')

In [4]:
from llm_chatbot_devs.utils.wikipidea import download_wikipidea_pages
from llm_chatbot_devs.utils.preprocessing import load_text_documents, text_splitter

In [5]:
download_wikipidea_pages()

In [6]:
documents = load_text_documents()
text_chunks = text_splitter(documents=documents)

In [7]:
len(text_chunks)

87

In [8]:
from llm_chatbot_devs.core.embeddings import get_instruct_embeddings

In [9]:
embeddings = get_instruct_embeddings()

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [9]:
from llm_chatbot_devs.utils.preprocessing import create_chroma_db

db = create_chroma_db(embeddings,text_chunks)


In [11]:
from llm_chatbot_devs.core.llms import create_wizard_lm_model_tokenizer, create_hf_pipeline


tokenizer, model = create_wizard_lm_model_tokenizer()
llm = create_hf_pipeline(model,tokenizer)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from llm_chatbot_devs.core.langchain import get_conversation_buffer_memory, get_conversational_chain

memory = get_conversation_buffer_memory(output_key='answer')
retriever = db.as_retriever(search_kwargs={"k": 3})
chain = get_conversational_chain(llm, retriever, memory,)

In [15]:
query = "The distance between Heidelberg and frankfurt?"
response = chain({"question": query})

In [18]:
response

{'question': 'The distance between Heidelberg and frankfurt?',
 'chat_history': [HumanMessage(content='The distance between Heidelberg and frankfurt?', additional_kwargs={}, example=False),
  AIMessage(content=' The distance between Heidelberg and Frankfurt is approximately 78 kilometers or 48 miles.', additional_kwargs={}, example=False)],
 'answer': ' The distance between Heidelberg and Frankfurt is approximately 78 kilometers or 48 miles.',
 'source_documents': [Document(page_content='Heidelberg (German: [ˈhaɪdl̩bɛʁk] (listen); Palatine German: Heidlberg) is a city in the German state of Baden-Württemberg, situated on the river Neckar in south-west Germany. As of the 2016 census, its population was 159,914, of which roughly a quarter consisted of students.Located about 78 km (48 mi) south of Frankfurt, Heidelberg is the fifth-largest city in Baden-Württemberg. Heidelberg is part of the densely populated Rhine-Neckar Metropolitan Region.\nHeidelberg University, founded in 1386, is Ge

In [19]:
from llm_chatbot_devs.utils.post_processing import wrap_text_preserve_newlines, process_llm_response
wrap_text_preserve_newlines(response['answer'])

' The distance between Heidelberg and Frankfurt is approximately 78 kilometers or 48 miles.'

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

In [ ]:
template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer. if you don't have the answer excuse for not having enough
information.

{context}

{chat_history}
Human: {query}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "query", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", output_key='answer', return_messages=True)


In [ ]:
chain = load_qa_chain(
    local_llm, chain_type="stuff", memory=memory, prompt=prompt,
)

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
chain_type_kwargs = {"prompt": prompt}

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                       memory=memory,
                                       chain_type_kwargs=chain_type_kwargs,
                                  return_source_documents=True)

In [ ]:
query = "History of heidelberg?"
# qa_chain({"query": query}, return_only_outputs=True)
qa_chain(query =  query)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Chain.__call__() got an unexpected keyword argument 'query'

In [ ]:
from langchain.chains import ConversationalRetrievalChain
qa = ConversationalRetrievalChain.from_llm(local_llm, retriever, memory=memory, return_source_documents=True)




In [ ]:
query = "History of heidelberg?"

In [ ]:
qa({"question": query}, return_only_outputs=True)

{'answer': 'Heidelberg has a long history dating back to the 5th century when the first settlers arrived in the area. It was during the 12th century that the town began to grow as the Holy Roman Empire established control over the region. In 1196, the first mention of Heidelberg was recorded in a document from the nearby Schönau Abbey. Over the next few hundred years, Heidelberg became an important center of commerce and culture, with the construction of the famous Heidelberg Castle in the early 13th century. During the Renaissance period, Heidelberg was a hub of intellectual activity, attracting many prominent figures such as Johann Wolfgang von Goethe and Friedrich Schiller. Today, Heidelberg is known as a major academic center, with the prestigious Heidelberg University being one of the oldest and most respected institutions of higher learning in Germany.',
 'source_documents': [Document(page_content='Heidelberg (German: [ˈhaɪdl̩bɛʁk] (listen); Palatine German: Heidlberg) is a city 

In [ ]:
query = "The distance between Heidelberg and frankfurt?"
qa({"question": query})

{'question': 'The distance between Heidelberg and frankfurt?',
 'chat_history': [HumanMessage(content='History of heidelberg?', additional_kwargs={}, example=False),
  AIMessage(content="Heidelberg has a long history dating back to the 5th century when the village of Bergheim was first mentioned in documents. Over time, the area was converted to Christianity and several monasteries were established, including the St. Michael's Monastery on the Heiligenberg hill and the Neuburg Monastery in the Neckar Valley. In the 12th century, the Bishopric of Worms extended its influence into the valley and founded the Schönau Abbey. Modern Heidelberg can trace its roots to these monasteries. The first reference to Heidelberg can be found in a document dated to 1196. In 1156, the castle and surrounding settlement were taken over by the House of Hohenstaufen, who became the Count Palatines of the Rhine.", additional_kwargs={}, example=False),
  HumanMessage(content='The distance between Heidelberg an

# ConversationalRetrievalChain with Question Answering with sources

In [ ]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain

question_generator = LLMChain(llm=local_llm, prompt=CONDENSE_QUESTION_PROMPT)
doc_chain = load_qa_with_sources_chain(local_llm, chain_type="map_reduce")

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
     memory=memory
)


In [ ]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], output_parser=None, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:', template_format='f-string', validate_template=True)

In [ ]:
query = "The distance between Heidelberg and frankfurt?"
chain({"question": query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Token indices sequence length is longer than the specified maximum sequence length for this model (1642 > 1024). Running this sequence through the model will result in indexing errors


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:166 in __call__                 │
│                                                                                                  │
│   163 │   │   │   )                                                                              │
│   164 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   165 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 166 │   │   │   raise e                                                                        │
│   167 │   │   run_manager.on_chain_end(outputs)                                                  │
│   168 │   │   final_outputs: Dict[str, Any] = self.prep_outputs(                                 │
│   169 │   │   │   inputs, outputs, return_only_outputs                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:160 in __call__                 │
│                                                                                                  │
│   157 │   │   )                                                                                  │
│   158 │   │   try:                                                                               │
│   159 │   │   │   outputs = (                                                                    │
│ ❱ 160 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   161 │   │   │   │   if new_arg_supported                                                       │
│   162 │   │   │   │   else self._call(inputs)                                                    │
│   163 │   │   │   )                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/conversational_retrieval/base.py:114 in │
│ _call                                                                                            │
│                                                                                                  │
│   111 │   │   new_inputs = inputs.copy()                                                         │
│   112 │   │   new_inputs["question"] = new_question                                              │
│   113 │   │   new_inputs["chat_history"] = chat_history_str                                      │
│ ❱ 114 │   │   answer = self.combine_docs_chain.run(                                              │
│   115 │   │   │   input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs         │
│   116 │   │   )                                                                                  │
│   117 │   │   output: Dict[str, Any] = {self.output_key: answer}                                 │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/langchain/chains/base.py:293 in run                      │
│                                                                                                  │
│   290 │   │   │   return self(args[0], callbacks=callbacks, tags=tags)[_output_key]              │
│   291 │   │                                                                                      │
│   292 │   │   if kwargs and not args:                                                            │
│ ❱ 293 │   │   │   return self(kwargs, callbacks=callbacks, 

In [ ]:
template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
chain_type_kwargs = {"prompt": prompt}

In [ ]:
query = "History of heidelberg?"

In [ ]:
docs = db.similarity_search(query)

In [ ]:
docs

[Document(page_content='Heidelberg (German: [ˈhaɪdl̩bɛʁk] (listen); Palatine German: Heidlberg) is a city in the German state of Baden-Württemberg, situated on the river Neckar in south-west Germany. As of the 2016 census, its population was 159,914, of which roughly a quarter consisted of students.Located about 78 km (48 mi) south of Frankfurt, Heidelberg is the fifth-largest city in Baden-Württemberg. Heidelberg is part of the densely populated Rhine-Neckar Metropolitan Region.\nHeidelberg University, founded in 1386, is Germany\'s oldest and one of Europe\'s most reputable universities. Heidelberg is a scientific hub in Germany and home to several internationally renowned research facilities adjacent to its university, including the European Molecular Biology Laboratory and four Max Planck Institutes. The city has also been a hub for the arts, especially literature, throughout the centuries, and it was designated a "City of Literature" by the UNESCO Creative Cities Network.', metada

In [ ]:
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': ' Heidelberg has a rich history dating back to the 5th century. The city was founded on the site of an ancient Celtic stronghold and grew to become an important center of learning and culture in the Middle Ages. During the Protestant Reformation, Heidelberg became a center of religious debate and controversy, hosting such figures as Martin Luther and John Calvin. Today, Heidelberg is known for its beautiful architecture, including the historic Old Town and the iconic Heidelberg Castle, which sits atop a hill overlooking the city.'}

In [ ]:
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': " Frankfurt am Main is a city in the German state of Hesse and the largest financial centre in continental Europe. It is home to the European Central Bank and many of the world's largest banks have their headquarters there."}

In [ ]:
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': ' Mannheim is a city in the German state of Baden-Württemberg, located on the River Rhine. It is the capital of the region of Nordbaden and is known for its baroque palace and gardens, as well as its industrial heritage.'}

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    sources = {}
    for source in llm_response["source_documents"]:
        sources[source.metadata['source']] = source.metadata['source']
    print("\n".join(sources))